In [1]:
# %load hpc_test_animate_true_color.py
#!/usr/bin/env python

import os
import sys
import glob
# import importlib
import gif_animate
# importlib.reload(gif_animate)
from gif_animate import *
from s3_file import s3_download
import ard_stac_search
# importlib.reload(ard_stac_search)
from ard_stac_search import Astac

print ("Number of arguments:", len(sys.argv), "arguments")
print ("Argument List:", str(sys.argv))

#arg=sys.argv[1]

arg='16_07_2022'
a = arg.split('_')

H=a[0]
V=a[1]
YEAR=a[2]

date_range_text = f"{YEAR}-01-01/{YEAR}-12-31"

#DATA_ROOT = '/caldera/projects/usgs/water/impd/butzer'
# DATA_ROOT = '/data2/butzer'
DATA_ROOT = '/home/ec2-user'

BROWSE = f'{DATA_ROOT}/browse_{H}_{V}'
ANIMATIONS = f'{DATA_ROOT}/animations'

os.makedirs(BROWSE, exist_ok=True)
os.makedirs(ANIMATIONS, exist_ok=True)

Number of arguments: 3 arguments
Argument List: ['/data/miniconda3/envs/nlcd2/lib/python3.11/site-packages/ipykernel_launcher.py', '-f', '/data/ec2-user/home/.local/share/jupyter/runtime/kernel-39942b04-3008-439b-8947-f10b37db3540.json']


In [2]:
stac_config = {}

stac_config['url'] = 'https://landsatlook.usgs.gov/stac-server'
stac_config['fav_collection'] = 'landsat-c2ard-sr'
stac_config['description'] = 'Landsat Collection 2 ARD Tiles Surface Reflection'


_stac = Astac(stac_config)

print(_stac.collection())
print(_stac.collections_list())
print(_stac.collections_df())

cloud_cover_pct_max = '99'

_assets = _stac.get_my_assets(None, H, V, date_range_text, cloud_cover_pct_max)

asset_df = _stac.fav_fields_df(_assets)

print(asset_df.head())
print(asset_df.columns)
print(_assets[0]['properties'].keys())

https://landsatlook.usgs.gov/stac-server
landsat-c2ard-sr
['landsat-c2l2-sr', 'landsat-c2l2-st', 'landsat-c2ard-st', 'landsat-c2l2alb-bt', 'landsat-c2l3-fsca', 'landsat-c2ard-bt', 'landsat-c2l1', 'landsat-c2l3-ba', 'landsat-c2l2alb-st', 'landsat-c2ard-sr', 'landsat-c2l2alb-sr', 'landsat-c2l2alb-ta', 'landsat-c2l3-dswe', 'landsat-c2ard-ta']
                    id  \
0      landsat-c2l2-sr   
1      landsat-c2l2-st   
2     landsat-c2ard-st   
3   landsat-c2l2alb-bt   
4    landsat-c2l3-fsca   
5     landsat-c2ard-bt   
6         landsat-c2l1   
7      landsat-c2l3-ba   
8   landsat-c2l2alb-st   
9     landsat-c2ard-sr   
10  landsat-c2l2alb-sr   
11  landsat-c2l2alb-ta   
12   landsat-c2l3-dswe   
13    landsat-c2ard-ta   

                                                                                                               title  
0                                                  Landsat Collection 2 Level-2 UTM Surface Reflectance (SR) Product  
1                            

In [3]:
asset_df.columns

Index(['product', 'date', 'grid', 'h', 'v', 'cc', 'fill', 'thumbnail',
       'reduced_resolution_browse', 'blue', 'green', 'red', 'nir08', 'swir16',
       'swir22', 'atmos_opacity', 'cloud_qa'],
      dtype='object')

In [4]:
print(_assets[0]['properties'].keys())

dict_keys(['start_datetime', 'end_datetime', 'datetime', 'platform', 'instruments', 'landsat:grid_horizontal', 'landsat:grid_vertical', 'landsat:grid_region', 'landsat:scene_count', 'eo:cloud_cover', 'landsat:cloud_shadow_cover', 'landsat:snow_ice_cover', 'landsat:fill', 'proj:epsg', 'proj:shape', 'proj:transform', 'proj:wkt2', 'created', 'updated'])


In [5]:
_assets[0]['properties']['landsat:grid_region']

'CU'

In [6]:
# # ref
# - https://github.com/Element84/geo-notebooks/blob/main/notebooks/planetary-computer-odc-stac-landsat.ipynb

for thumb in asset_df['reduced_resolution_browse']:
    print(thumb)

s3url = asset_df['reduced_resolution_browse'][0]
print(s3url)

s3_download(s3url, BROWSE)

s3://usgs-landsat-ard/collection02/oli-tirs/2022/CU/016/007/LC09_CU_016007_20221229_20230319_02/LC09_CU_016007_20221229_20230319_02_thumb_large.jpeg
s3://usgs-landsat-ard/collection02/oli-tirs/2022/CU/016/007/LC08_CU_016007_20221228_20230107_02/LC08_CU_016007_20221228_20230107_02_thumb_large.jpeg
s3://usgs-landsat-ard/collection02/oli-tirs/2022/CU/016/007/LC09_CU_016007_20221227_20230319_02/LC09_CU_016007_20221227_20230319_02_thumb_large.jpeg
s3://usgs-landsat-ard/collection02/oli-tirs/2022/CU/016/007/LC09_CU_016007_20221220_20230320_02/LC09_CU_016007_20221220_20230320_02_thumb_large.jpeg
s3://usgs-landsat-ard/collection02/oli-tirs/2022/CU/016/007/LC09_CU_016007_20221213_20230414_02/LC09_CU_016007_20221213_20230414_02_thumb_large.jpeg
s3://usgs-landsat-ard/collection02/oli-tirs/2022/CU/016/007/LC09_CU_016007_20221211_20230321_02/LC09_CU_016007_20221211_20230321_02_thumb_large.jpeg
s3://usgs-landsat-ard/collection02/oli-tirs/2022/CU/016/007/LC08_CU_016007_20221205_20230116_02/LC08_CU_01

In [7]:
# tqdm here perhaps?
DOWN=False
if DOWN:
    for thumb in asset_df['reduced_resolution_browse']:
        s3url = thumb
        s3_download(s3url, BROWSE)
    #print(thumb)

In [8]:
#! mkdir /data2/butzer/animations

frame_folder=BROWSE
output_gif_name=f'{ANIMATIONS}/{YEAR}_{H}_{V}.gif'

for image in glob.glob(f"{frame_folder}/*.jpeg"):
    print(image)
    em_str = image.split('/')[-1].replace('_02_thumb_large.jpeg', '')
    emboss_image(image, em_str)

/home/ec2-user/browse_16_07/LC08_CU_016007_20221002_20221016_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/LC09_CU_016007_20221229_20230319_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/LC09_CU_016007_20221008_20230329_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/LC08_CU_016007_20221228_20230107_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/LC08_CU_016007_20220923_20221002_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/LC09_CU_016007_20221227_20230319_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/LC09_CU_016007_20220924_20230331_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/LC09_CU_016007_20221220_20230320_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/LC09_CU_016007_20220922_20230331_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/LC09_CU_016007_20221213_20230414_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/LC08_CU_016007_20220916_20220925_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/LC09_CU_016007_20221211_20230321_02_thumb_large.jpeg
/home/ec2-user/browse_16_07/

In [9]:
print('Building the animation ...')
make_gif_from_jpeg(frame_folder, output_gif_name)
print(f'SUCCESS {output_gif_name} created!')

Building the animation ...
SUCCESS /home/ec2-user/animations/2022_16_07.gif created!


In [10]:
print('Building the mp4 animation ...')

output_mpeg_name=f'{ANIMATIONS}/{YEAR}_{H}_{V}.mpg'
#output_mpeg_name=f'{ANIMATIONS}/{YEAR}_{H}_{V}.mp4'
make_mpeg_from_jpeg(frame_folder, output_mpeg_name)
print(f'SUCCESS {output_gif_name} created!')


# print(frame_folder)

#print(asset_df.head())

Building the mp4 animation ...


KeyError: 'MPEG'

In [ ]:
#! rm /home/ec2-user/browse_16_07/*

In [14]:
!ffmpeg -f gif -i 2022_16_07.gif outfile.mp4

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


In [ ]:
! conda install ffmpeg

In [15]:
! ffmpeg -i 2022_16_07.gif -movflags faststart -pix_fmt yuv420p -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" video.mp4

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
